In [3786]:
import pandas as pd
import numpy as np
import re
from datetime import timedelta


## 1. PASOS DE LA DEPURACION
- Exportacion de csv a trabajar
- Visualizacion de los datos transformandolos a un DataFrame y asi tener entendimiento del contenido a trabajar.
- Limpieza de datos por columna y verificacion de tipo de dato cada una tiene 
- Se debe identificar errores: Detectar datos inválidos, incompletos, duplicados o irrelevantes.


- Exportación del fichero CSV y transformacion a un DataFrame 

In [3787]:
retail2_df = pd.read_csv('retail2.csv')

- Reconocimiento del contenido de CSV

In [3788]:
# Para mostrar todas las filas y columnas
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
retail2_df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,CustomerName,Email,Address,PhoneNumber,Category,Supplier,StockLevel,Discount,SaleChannel,ReturnStatus,ProductWeight,ProductDimensions,ShippingCost,SalesRegion,PromotionCode,PaymentMethod
0,536578.0,84969,"[""description"": ""BOX OF 6 ASSORTED COLOUR TEAS...",6,12/1/2010 12:28,4.25,17763,United Kingdom,David Johnson,david.johnson@mail.com,450 Ash St,555-734-2925,Beauty,Supplier A,853,17.28,Online,Not Returned,3.81,37x38x83 cm,7.14,North America,SALE15,Bank Transfer
1,536446.0,21756,DOORMAT NEW ENGLAND,100,12/1/2010 10:16,795.0,15939,United Kingdom,Henry Williams,henry.williams@test.org,815 Maple St,555-782-8115,Toys,Supplier A,910,9.08,Online,Not Returned,9.51,8x65x86 cm,12.48,Asia,SALE15,Bank Transfer
2,536633.0,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 13:23,1.85,12295,United Kingdom,Jane Brown,jane.brown@mail.com,253 High St,555-936-2229,Sports,Supplier D,578,45.42,In-Store,Returned,7.35,17x71x89 cm,14.27,North America,DISCOUNT5,Bank Transfer
3,536522.0,22111,"{""description"": ""SCANDINAVIAN REDS RIBBONS""}",10,12/1/2010 11:32,1.65,15685,United Kingdom,Frank Johnson,frank.johnson@example.com,883 Elm St,555-242-3875,Beauty,Supplier C,75,29.17,Online,Returned,6.03,45x4x36 cm,15.54,Asia,NaN,PayPal
4,NaN,22634,"{""description"": ""BAKING SET 9 PIECE RETROSPOT""}",6,12/1/2010 11:07,4.95,11696,United Kingdom,Eva Smith,eva.smith@mail.com,709 Pine St,555-412-6954,Home & Garden,Supplier C,435,20.04,Online,Not Returned,1.64,70x31x19 cm,13.39,Australia,PROMO10,Bank Transfer


## Limpieza de datos por columna 

## Columna Invoice :

- Rellenar los valores Nulos con un valor especifico con 0:Realizado en el DataFrame retail2_df
- Cambiar los datos flotantes a números enteros:Realizado en el DataFrame retail2_df

1. vamos a verificar que tipo de dato la columna InvoiceNo, esta debe es de tipo entero 

In [3789]:
#Verificando que tipo de dato es la columna 
print(retail2_df['InvoiceNo'].dtype)

float64


In [3790]:
# Rellenar los valores NaN con 0 y convertir a int
retail2_df['InvoiceNo'] = retail2_df['InvoiceNo'].fillna(0).astype(int)

# Verifica el tipo de dato
print(retail2_df['InvoiceNo'].dtype)

int64


## Columna StockCode:
- Verificación del tipo del dato  
- Visualización de los datos para realizar los cambios correspondientes.
- Eliminación de espacios en blanco al principio y al final de los valores en la columna
- Movimientos de datos a las columna correspontiendes que no estan relacionados con la columna. 
- Identificacón de los datos inválidos. 
- Relleno de los datos faltantes.

- Verificación tipo de dato

In [3791]:
print(retail2_df['StockCode'].dtype)

object


- Visualización de los datos ordenadamente 

In [3792]:
retail2_df=retail2_df.sort_values(by='StockCode')
print(retail2_df['StockCode'].unique())

['21723' '21724' '21730' '21731' '21754' '21755' '21756' '21777' '21914'
 '22111' '22112' '22139' '22310' '22382' '22384' '22386' '22469' '22570'
 '22571' '22622' '22623' '22624' '22632' '22633' '22634' '22725' '22745'
 '22748' '22749' '22751' '22752' '22754' '22960' '71053' '84029E' '84029G'
 '84406B' '84879' '84907A' '84907B' '84907C' '84969' '84988' '85099B'
 '85123A' 'BOX OF 6 ASSORTED COLOUR TEASPOONS' 'HAND WARMER RED POLKA DOT'
 'JAM MAKING SET WITH JARS' 'LOVE BUILDING BLOCK WORD'
 'RED  HARMONICA IN BOX' 'RED HARMONICA IN BOX'
 'SCANDINAVIAN REDS RIBBONS' 'SET 2 TEA TOWELS I LOVE LONDON'
 'STRIPED CHARLIE+LOLA CHARLOTTE BAG'
 '{"description": ["RED WOOLLY HOTTIE WHITE HEART.: details"]}']


- Eliminación de espacios en blanco al principio y al final de los valores en la columna

In [3793]:
retail2_df['StockCode'] = retail2_df['StockCode'].str.strip()

# Mostrar el DataFrame para verificar los cambios
print(retail2_df['StockCode'].head())

291    21723
34     21723
165    21723
40     21723
151    21723
Name: StockCode, dtype: object


- Movimientos de datos a las columna correspontiendes que no estan relacionados con la columna.
En este caso se encotro datos de la columna Description por lo tanto se transladan a ella.  

In [3794]:
# Mover los datos de StockCode a Description si StockCode contiene texto y no comienza con un número
mask = retail2_df['StockCode'].apply(lambda x: isinstance(x, str) and not x[0].isdigit())
retail2_df.loc[mask, 'Description'] = retail2_df.loc[mask, 'StockCode']

# Limpiar la columna StockCode para eliminar los datos que se movieron a Description
retail2_df.loc[mask, 'StockCode'] = ''

# Ordenar la salida por el índice
retail2_df_sorted = retail2_df.sort_index()

# Mostrar las columnas StockCode y Description
print(retail2_df_sorted[['StockCode', 'Description']])

    StockCode                                        Description
0       84969  ["description": "BOX OF 6 ASSORTED COLOUR TEAS...
1       21756                                DOORMAT NEW ENGLAND
2       22632                          HAND WARMER RED POLKA DOT
3       22111       {"description": "SCANDINAVIAN REDS RIBBONS"}
4       22634    {"description": "BAKING SET 9 PIECE RETROSPOT"}
5       22960      {"description": ["JAM MAKING SET WITH JARS"]}
6       22139                    RETROSPOT TEA SET CERAMIC 11 PC
7       22112                         CHOCOLATE HOT WATER BOTTLE
8       22634                       BAKING SET 9 PIECE RETROSPOT
9       22749                                                NaN
10      22632                          HAND WARMER RED POLKA DOT
11      22112                         CHOCOLATE HOT WATER BOTTLE
12      21724                               RED HARMONICA IN BOX
13      71053                                WHITE METAL LANTERN
14     84029G            

- Confirmación de los datos luego del traslado realizado.

In [3795]:
retail2_df=retail2_df.sort_values(by='StockCode')
print(retail2_df['StockCode'].unique())

['' '21723' '21724' '21730' '21731' '21754' '21755' '21756' '21777'
 '21914' '22111' '22112' '22139' '22310' '22382' '22384' '22386' '22469'
 '22570' '22571' '22622' '22623' '22624' '22632' '22633' '22634' '22725'
 '22745' '22748' '22749' '22751' '22752' '22754' '22960' '71053' '84029E'
 '84029G' '84406B' '84879' '84907A' '84907B' '84907C' '84969' '84988'
 '85099B' '85123A']


- Identificación de los datos inválidos.

In [3796]:
# Identificar los datos inválidos en la columna StockCode, incluyendo NaN y vacíos
invalid_data_stockcode = retail2_df[retail2_df['StockCode'].isnull() | (retail2_df['StockCode'] == '') | (retail2_df['StockCode'].str.lower() == 'nan')]

# Mostrar los datos inválidos y la cantidad
print(invalid_data_stockcode[['StockCode','Description']])
print(f"Cantidad de datos inválidos: {invalid_data_stockcode.shape[0]}")

    StockCode                                        Description
314            {"description": ["RED WOOLLY HOTTIE WHITE HEAR...
49                                SET 2 TEA TOWELS I LOVE LONDON
322                                    SCANDINAVIAN REDS RIBBONS
316                                         RED HARMONICA IN BOX
298                                        RED  HARMONICA IN BOX
292                                     LOVE BUILDING BLOCK WORD
369                                     JAM MAKING SET WITH JARS
272                                    HAND WARMER RED POLKA DOT
136                           BOX OF 6 ASSORTED COLOUR TEASPOONS
213                           STRIPED CHARLIE+LOLA CHARLOTTE BAG
Cantidad de datos inválidos: 10


Los datos inválidos corresponden a campos vacíos sin información.
Por lo anterior se rellena los valores faltantes en la columna ‘StockCode’ utilizando un diccionario de referencia basado en la columna ‘Description’  y luego identificaremos y tendremos los datos que aún son inválidos.

In [3797]:
#Crear un diccionario de referencia
reference_dict_stockcode = retail2_df.dropna(subset=['StockCode']).set_index('Description')['StockCode'].to_dict()

# Función para rellenar los valores faltantes
def fill_missing_descriptions(row):
    if pd.isna(row['StockCode']) or row['StockCode'] == '':
        return reference_dict_stockcode.get(row['Description'], row['StockCode'])
    return row['StockCode']

# Aplicar la función al DataFrame
retail2_df['StockCode'] = retail2_df.apply(fill_missing_descriptions, axis=1)

# Identificar datos inválidos
invalid_data_stockcode = retail2_df[retail2_df['StockCode'].isnull() | (retail2_df['StockCode'] == '') | (retail2_df['StockCode'].str.lower() == 'nan')]

# Mostrar los datos inválidos y la cantidad
print(invalid_data_stockcode[['StockCode', 'Description']])
print(f"Cantidad de datos inválidos: {invalid_data_stockcode.shape[0]}")

    StockCode                                        Description
314            {"description": ["RED WOOLLY HOTTIE WHITE HEAR...
Cantidad de datos inválidos: 1


## Explicación del código

- 1. Crear un diccionario de referencia:
retail2_df.dropna(subset=['StockCode']): Elimina las filas donde StockCode es NaN.
.set_index('Description')['StockCode']: Establece Description como índice y selecciona la columna StockCode.
.to_dict(): Convierte el resultado en un diccionario donde las claves son las descripciones y los valores son los códigos de stock.

- 2. Definir una función para rellenar los valores faltantes:
if pd.isna(row['StockCode']) or row['StockCode'] == '': Verifica si StockCode es NaN o está vacío.
return reference_dict_stockcode.get(row['Description'], row['StockCode']): Busca en el diccionario reference_dict_stockcode el StockCode correspondiente a la Description. Si no encuentra una coincidencia, devuelve el valor original de StockCode.
return row['StockCode']: Si StockCode no es NaN ni está vacío, devuelve el valor original.

- 3. Aplicar la función al DataFrame:
retail2_df.apply(fill_missing_descriptions, axis=1): Aplica la función fill_missing_descriptions a cada fila del DataFrame retail2_df. axis=1 indica que la función se aplica a lo largo de las filas.

- 4. retail2_df['StockCode'].isnull(): Verifica si StockCode es NaN.
(retail2_df['StockCode'] == ''): Verifica si StockCode está vacío.
(retail2_df['StockCode'].str.lower() == 'nan'): Verifica si StockCode contiene la cadena 'nan' (en minúsculas).
retail2_df[...]: Filtra el DataFrame para incluir solo las filas que cumplen alguna de estas condiciones.


Se observa que aun queda un dato inválido que al momento de corregir los datos correspondientes a la columna Description se solucionara.

## Columna Description 

- Verificación del tipo del dato  
- Visualización de los datos para realizar los cambios correspondientes.
- Eliminación de espacios en blanco al principio y al final de los valores en la columna
- Función para limpiar y unificar las descripciones :Eliminación de palabras y caracteres no correspondientes a la descripcion.
- Se realiza los cambios minimos con las descripciones especificas a las cuales la función no reconocio. 
- Identificar los datos inválidos en la columna Description, incluyendo NaN y vacíos
- Rellenar los valores faltantes en la columna ‘Description’ utilizando un diccionario de referencia basado en la columna ‘StockCode’.

- Verificación del tipo del dato 

In [3798]:
print(retail2_df['Description'].dtype)

object


- Visualización de los datos para realizar los cambios correspondientes.

In [3799]:
# Ordenar el DataFrame por la columna 'Description'
retail2_df = retail2_df.sort_values(by='Description')

# Obtener los valores únicos y contarlos
unique_descriptions = retail2_df['Description'].unique()
unique_count = retail2_df['Description'].nunique()

# Mostrar los valores únicos y la cantidad
print(unique_descriptions)
print(f"Cantidad de valores únicos: {unique_count}")

['  BAKING SET 9 PIECE RETROSPOT  ' '  BOX OF VINTAGE ALPHABET BLOCKS  '
 '  BOX OF VINTAGE JIGSAW BLOCKS  ' '  CHOCOLATE HOT WATER BOTTLE  '
 '  CREAM CUPID HEARTS COAT HANGER  ' '  HAND WARMER RED POLKA DOT  '
 '  HAND WARMER UNION JACK  ' '  HOME BUILDING BLOCK WORD  '
 '  JUMBO BAG RED RETROSPOT  ' '  PACK OF 12 BLUE RETROSPOT TISSUES  '
 '  PACK OF 6 SMALL FRUIT STRAWS  ' "  POPPY'S PLAYHOUSE BEDROOM  "
 "  POPPY'S PLAYHOUSE KITCHEN  " '  RED HARMONICA IN BOX  '
 '  RED TOADSTOOL LED NIGHT LIGHT  ' '  RETROSPOT TEA SET CERAMIC 11 PC  '
 '  WHITE METAL LANTERN  ' '  nan  ' '***BAKING SET 9 PIECE RETROSPOT***'
 '***CHOCOLATE HOT WATER BOTTLE***' '***DOORMAT NEW ENGLAND***'
 '***FELTCRAFT PRINCESS CHARLOTTE DOLL***'
 '***HAND WARMER RED POLKA DOT***' '***IVORY KNITTED MUG COSY***'
 '***JAM MAKING SET WITH JARS***' "***POPPY'S PLAYHOUSE BEDROOM***"
 "***POPPY'S PLAYHOUSE KITCHEN***" '***RED GLASS CANDLE HOLDER STAR***'
 '***RED TOADSTOOL LED NIGHT LIGHT***'
 '***RED WOOLLY HOTTIE WHIT

- Eliminación de espacios en blanco al principio y al final de los valores en la columna

In [3800]:
retail2_df['Description'] = retail2_df['Description'].str.strip()

# Mostrar el DataFrame para verificar los cambios
print(retail2_df['Description'].head())

70       BAKING SET 9 PIECE RETROSPOT
29     BOX OF VINTAGE ALPHABET BLOCKS
388      BOX OF VINTAGE JIGSAW BLOCKS
102        CHOCOLATE HOT WATER BOTTLE
198    CREAM CUPID HEARTS COAT HANGER
Name: Description, dtype: object


- Función para limpiar y unificar las descripciones según su mayoria de coincidencias y la eliminación de datos no correspondientes 

In [3801]:
# Función para limpiar y unificar las descripciones
def clean_description(desc):
    # Eliminar las palabras "description" y "details" si están presentes
    desc = re.sub(r'description|details', '', desc, flags=re.IGNORECASE)
    # Eliminar caracteres especiales y convertir a mayúsculas
    desc = re.sub(r'[^a-zA-Z0-9\s]', '', desc).upper()
    # Eliminar espacios adicionales
    desc = re.sub(r'\s+', ' ', desc).strip()
    return desc

# Aplicar la función de limpieza a la columna 'Description'
retail2_df['Description'] = retail2_df['Description'].apply(lambda x: clean_description(x) if isinstance(x, str) else x)

# Reemplazar "INVALID123", 0 y NaN con cadenas vacías
retail2_df['Description'] = retail2_df['Description'].replace(['INVALID123', 0, np.nan], '')

# Ordenar el DataFrame por la columna 'Description'
retail2_df = retail2_df.sort_values(by='Description')

# Imprimir los valores únicos y la cantidad de valores únicos
print(retail2_df['Description'].unique())
print(f"Cantidad de valores únicos: {retail2_df['Description'].nunique()}")



['' 'ASSORTED COLOUR BIRD ORNAMENT' 'BAKING SET 9 PIECE RETROSPOT'
 'BLUE HARMONICA IN BOX' 'BOX OF 6 ASSORTED COLOUR TEASPOONS'
 'BOX OF VINTAGE ALPHABET BLOCKS' 'BOX OF VINTAGE JIGSAW BLOCKS'
 'CHOCOLATE HOT WATER BOTTLE' 'COFFEE MUG APPLES DESIGN'
 'CREAM CUPID HEARTS COAT HANGER' 'DOORMAT NEW ENGLAND'
 'FELTCRAFT PRINCESS CHARLOTTE DOLL' 'GLASS JAR DAISY FRESH COTTON WOOL'
 'GLASS STAR FROSTED TLIGHT HOLDER' 'HAND WARMER RED POLKA DOT'
 'HAND WARMER UNION JACK' 'HANGING STAR WITH BELLS'
 'HEART OF WICKER LARGE' 'HOME BUILDING BLOCK WORD'
 'IVORY KNITTED MUG COSY' 'JAM MAKING SET WITH JARS'
 'JUMBO BAG PINK POLKADOT' 'JUMBO BAG RED RETROSPOT'
 'KNITTED UNION FLAG HOT WATER BOTTLE' 'LOVE BUILDING BLOCK WORD' 'NAN'
 'PACK OF 12 BLUE RETROSPOT TISSUES' 'PACK OF 12 PINK POLKADOT TISSUES'
 'PACK OF 12 RED RETROSPOT TISSUES' 'PACK OF 6 SMALL FRUIT STRAWS'
 'POPPYS PLAYHOUSE BEDROOM' 'POPPYS PLAYHOUSE KITCHEN'
 'RECIPE BOX WITH METAL HEART' 'RED GLASS CANDLE HOLDER STAR'
 'RED HARMONICA IN

- Se realiza los cambios minimos con las descripciones no válidas. 

In [3802]:
# Usar loc para identificar y reemplazar el valor específico
retail2_df.loc[retail2_df['Description'] == 'SET OF 7 BABUSHKA NESTING BOXES', 'Description'] = 'SET 7 BABUSHKA NESTING BOXES'

# Verificar el cambio
print(retail2_df['Description'].unique())
print(f"Cantidad de valores únicos: {retail2_df['Description'].nunique()}")



['' 'ASSORTED COLOUR BIRD ORNAMENT' 'BAKING SET 9 PIECE RETROSPOT'
 'BLUE HARMONICA IN BOX' 'BOX OF 6 ASSORTED COLOUR TEASPOONS'
 'BOX OF VINTAGE ALPHABET BLOCKS' 'BOX OF VINTAGE JIGSAW BLOCKS'
 'CHOCOLATE HOT WATER BOTTLE' 'COFFEE MUG APPLES DESIGN'
 'CREAM CUPID HEARTS COAT HANGER' 'DOORMAT NEW ENGLAND'
 'FELTCRAFT PRINCESS CHARLOTTE DOLL' 'GLASS JAR DAISY FRESH COTTON WOOL'
 'GLASS STAR FROSTED TLIGHT HOLDER' 'HAND WARMER RED POLKA DOT'
 'HAND WARMER UNION JACK' 'HANGING STAR WITH BELLS'
 'HEART OF WICKER LARGE' 'HOME BUILDING BLOCK WORD'
 'IVORY KNITTED MUG COSY' 'JAM MAKING SET WITH JARS'
 'JUMBO BAG PINK POLKADOT' 'JUMBO BAG RED RETROSPOT'
 'KNITTED UNION FLAG HOT WATER BOTTLE' 'LOVE BUILDING BLOCK WORD' 'NAN'
 'PACK OF 12 BLUE RETROSPOT TISSUES' 'PACK OF 12 PINK POLKADOT TISSUES'
 'PACK OF 12 RED RETROSPOT TISSUES' 'PACK OF 6 SMALL FRUIT STRAWS'
 'POPPYS PLAYHOUSE BEDROOM' 'POPPYS PLAYHOUSE KITCHEN'
 'RECIPE BOX WITH METAL HEART' 'RED GLASS CANDLE HOLDER STAR'
 'RED HARMONICA IN

In [3803]:
retail2_df['Description'] = retail2_df['Description'].replace({
    'STRIPED CHARLIELOLA CHARLOTTE BAG': 'STRIPED CHARLIE+LOLA CHARLOTTE BAG',
    'WHITE HANGING HEART TLIGHT HOLDER': 'WHITE HANGING HEART T-LIGHT HOLDER'
})

# Verificar los cambios
print(retail2_df['Description'].unique())

['' 'ASSORTED COLOUR BIRD ORNAMENT' 'BAKING SET 9 PIECE RETROSPOT'
 'BLUE HARMONICA IN BOX' 'BOX OF 6 ASSORTED COLOUR TEASPOONS'
 'BOX OF VINTAGE ALPHABET BLOCKS' 'BOX OF VINTAGE JIGSAW BLOCKS'
 'CHOCOLATE HOT WATER BOTTLE' 'COFFEE MUG APPLES DESIGN'
 'CREAM CUPID HEARTS COAT HANGER' 'DOORMAT NEW ENGLAND'
 'FELTCRAFT PRINCESS CHARLOTTE DOLL' 'GLASS JAR DAISY FRESH COTTON WOOL'
 'GLASS STAR FROSTED TLIGHT HOLDER' 'HAND WARMER RED POLKA DOT'
 'HAND WARMER UNION JACK' 'HANGING STAR WITH BELLS'
 'HEART OF WICKER LARGE' 'HOME BUILDING BLOCK WORD'
 'IVORY KNITTED MUG COSY' 'JAM MAKING SET WITH JARS'
 'JUMBO BAG PINK POLKADOT' 'JUMBO BAG RED RETROSPOT'
 'KNITTED UNION FLAG HOT WATER BOTTLE' 'LOVE BUILDING BLOCK WORD' 'NAN'
 'PACK OF 12 BLUE RETROSPOT TISSUES' 'PACK OF 12 PINK POLKADOT TISSUES'
 'PACK OF 12 RED RETROSPOT TISSUES' 'PACK OF 6 SMALL FRUIT STRAWS'
 'POPPYS PLAYHOUSE BEDROOM' 'POPPYS PLAYHOUSE KITCHEN'
 'RECIPE BOX WITH METAL HEART' 'RED GLASS CANDLE HOLDER STAR'
 'RED HARMONICA IN

 - Identificación de los datos inválidos en la columna Description, incluyendo NaN y vacíos

In [3804]:
# Identificar los datos inválidos en la columna Description, incluyendo NaN y vacíos
invalid_data = retail2_df[retail2_df['Description'].isnull() | (retail2_df['Description'] == '') | (retail2_df['Description'].str.lower() == 'nan')]

# Mostrar los datos inválidos y la cantidad
print(invalid_data[['StockCode','Description']])
print(f"Cantidad de datos inválidos: {invalid_data.shape[0]}")

    StockCode Description
208    85123A            
259    85123A            
252     22623            
251     22632            
37      22622            
186     22382            
168     22634            
315     22139            
274     21755            
390     21723            
360     84879            
227     21724            
203     21724            
345     21724            
239     21755            
309     21756            
145     22960            
223     21730            
9       22749            
182     22745            
43      22634            
319     22634            
276     22633            
130     22632            
169     22752            
401     22386            
115     22310            
32      21914            
243     22632            
348     21914         NAN
Cantidad de datos inválidos: 30


- Rellenar los valores faltantes en la columna ‘Description’ utilizando un diccionario de referencia basado en la columna ‘StockCode’.

In [3805]:
# Crear un diccionario de referencia
reference_dict = retail2_df.dropna(subset=['Description']).set_index('StockCode')['Description'].to_dict()

# Función para rellenar los valores faltantes
def fill_missing_description(row):
    if pd.isna(row['Description']) or row['Description'] in ['', '0']:
        return reference_dict.get(row['StockCode'], row['Description'])
    return row['Description']

# Aplicar la función al DataFrame
retail2_df['Description'] = retail2_df.apply(fill_missing_description, axis=1)

# Ordenar el DataFrame por la columna 'Description'
retail2_df = retail2_df.sort_values(by='Description')

# Imprimir los valores únicos y la cantidad de valores únicos
print(retail2_df['Description'].unique())
print(f"Cantidad de valores únicos: {retail2_df['Description'].nunique()}")

['' 'ASSORTED COLOUR BIRD ORNAMENT' 'BAKING SET 9 PIECE RETROSPOT'
 'BLUE HARMONICA IN BOX' 'BOX OF 6 ASSORTED COLOUR TEASPOONS'
 'BOX OF VINTAGE ALPHABET BLOCKS' 'BOX OF VINTAGE JIGSAW BLOCKS'
 'CHOCOLATE HOT WATER BOTTLE' 'COFFEE MUG APPLES DESIGN'
 'CREAM CUPID HEARTS COAT HANGER' 'DOORMAT NEW ENGLAND'
 'FELTCRAFT PRINCESS CHARLOTTE DOLL' 'GLASS JAR DAISY FRESH COTTON WOOL'
 'GLASS STAR FROSTED TLIGHT HOLDER' 'HAND WARMER RED POLKA DOT'
 'HAND WARMER UNION JACK' 'HANGING STAR WITH BELLS'
 'HEART OF WICKER LARGE' 'HOME BUILDING BLOCK WORD'
 'IVORY KNITTED MUG COSY' 'JAM MAKING SET WITH JARS'
 'JUMBO BAG PINK POLKADOT' 'JUMBO BAG RED RETROSPOT'
 'KNITTED UNION FLAG HOT WATER BOTTLE' 'LOVE BUILDING BLOCK WORD' 'NAN'
 'PACK OF 12 BLUE RETROSPOT TISSUES' 'PACK OF 12 PINK POLKADOT TISSUES'
 'PACK OF 12 RED RETROSPOT TISSUES' 'PACK OF 6 SMALL FRUIT STRAWS'
 'POPPYS PLAYHOUSE BEDROOM' 'POPPYS PLAYHOUSE KITCHEN'
 'RECIPE BOX WITH METAL HEART' 'RED GLASS CANDLE HOLDER STAR'
 'RED HARMONICA IN

In [3806]:
print(retail2_df[['InvoiceNo','StockCode','Description']])

     InvoiceNo StockCode                          Description
186     536407     22382                                     
352     536473     84879        ASSORTED COLOUR BIRD ORNAMENT
96      536734     84879        ASSORTED COLOUR BIRD ORNAMENT
257     536573     84879        ASSORTED COLOUR BIRD ORNAMENT
318     536673     84879        ASSORTED COLOUR BIRD ORNAMENT
80      536634     84879        ASSORTED COLOUR BIRD ORNAMENT
189     536420     84879        ASSORTED COLOUR BIRD ORNAMENT
360     536367     84879        ASSORTED COLOUR BIRD ORNAMENT
28      536534     84879        ASSORTED COLOUR BIRD ORNAMENT
296     536697     22634         BAKING SET 9 PIECE RETROSPOT
425     536660     22634         BAKING SET 9 PIECE RETROSPOT
279     536709     22634         BAKING SET 9 PIECE RETROSPOT
380     536444     22634         BAKING SET 9 PIECE RETROSPOT
167          0     22634         BAKING SET 9 PIECE RETROSPOT
39      536448     22634         BAKING SET 9 PIECE RETROSPOT
8       

In [3807]:

print(retail2_df.loc[251, ['InvoiceNo', 'StockCode','Description', 'InvoiceDate']])

InvoiceNo                              0
StockCode                          22632
Description    HAND WARMER RED POLKA DOT
InvoiceDate              12/1/2010 12:53
Name: 251, dtype: object


- Posterior a lo anterior se identifica nuevamente los datos inválidos

In [3808]:
# Identificar los datos inválidos en la columna Description, incluyendo NaN y vacíos
invalid_data = retail2_df[retail2_df['Description'].isnull() | (retail2_df['Description'] == '') | (retail2_df['Description'].str.lower() == 'nan')]

# Mostrar los datos inválidos y la cantidad
print(invalid_data[['StockCode','Description']])
print(f"Cantidad de datos inválidos: {invalid_data.shape[0]}")

    StockCode Description
186     22382            
348     21914         NAN
32      21914         NAN
Cantidad de datos inválidos: 3


In [3809]:
# Actualizar el código 22382 a 22386 y la descripción correspondiente
retail2_df.loc[retail2_df['StockCode'] == 22382, 'StockCode'] = 22386
retail2_df.loc[retail2_df['StockCode'] == 22386, 'Description'] = 'JUMBO BAG PINK POLKADOT'

# Actualizar la descripción para los índices 348 y 32 con el código 21914
indices_a_actualizar = [348, 32]
retail2_df.loc[indices_a_actualizar, 'Description'] = 'BLUE HARMONICA IN BOX'



In [3810]:
# Actualizar el código 22382 a 22386 en el índice 186
retail2_df.loc[186, 'StockCode'] = 22386
retail2_df.loc[186, 'Description'] = 'JUMBO BAG PINK POLKADOT'

In [3811]:
# Identificar los datos inválidos en la columna Description, incluyendo NaN y vacíos
invalid_data = retail2_df[retail2_df['Description'].isnull() | (retail2_df['Description'] == '') | (retail2_df['Description'].str.lower() == 'nan')]

# Mostrar los datos inválidos y la cantidad
print(invalid_data[['StockCode','Description']])
print(f"Cantidad de datos inválidos: {invalid_data.shape[0]}")

Empty DataFrame
Columns: [StockCode, Description]
Index: []
Cantidad de datos inválidos: 0


## Limpieza de datos: columna InvoiceDate

- Verificación de datos
- Eliminación de texto no correspondientes al formato de fechas : palabra 'Date'.
- Convertir la columna InvoiceDate a datetime con el formato correcto que es dia, mes, año y hora. 
- Defenición de fechas: En este caso fueron las ventas correspondiente a un solo dia  12/01/2010 y con su respectiva hora
- Verificación de datos inválidos:Encontramos fechas correspondiente a años como 2050, 1900, dias no correspondientes a la fecha a trabajar 30/02/2010, formatos de fechas no válidas como mes, dia, año y caracteres no correspondientes como -, se obtiene columnas vacías. 
- Correción de los datos inválidos mencionados anteriormente.
- Limpiar y corregir los números de factura y las fechas en la columna ‘InvoiceDate’, asegurando que todas las facturas tengan fechas válidas y consistentes: Para los datos sin fecha se tiene encuanta el consecutivo de la hora tomando de refencia la columna de las facturas existentes en la columna InvoiceDate.
- Corrección de los valores de ‘InvoiceNo’ y ‘InvoiceDate’ para ciertas filas específicas, asegurando que las fechas y números de factura sean correctos y estén en el formato adecuado. 

- Verificación de datos 

In [3812]:
# Verificación de datos y se observan en orden 
retail2_df = retail2_df.sort_values(by='InvoiceDate') 
print(retail2_df['InvoiceDate'].unique())

['01-12-2010 09:47' '01-12-2010 10:09' '01-12-2010 11:27'
 '01-12-2010 13:42' '01/01/1900 00:00' '01/01/2050 00:00'
 '01/12/2010 25:61' '12/01/2010 09:08' '12/01/2010 10:41'
 '12/01/2010 10:52' '12/01/2010 12:22' '12/01/2010 14:55'
 '12/1/2010 10:00' '12/1/2010 10:01' '12/1/2010 10:03' '12/1/2010 10:04'
 '12/1/2010 10:05' '12/1/2010 10:06' '12/1/2010 10:07' '12/1/2010 10:08'
 '12/1/2010 10:10' '12/1/2010 10:12' '12/1/2010 10:13' '12/1/2010 10:15'
 '12/1/2010 10:16' '12/1/2010 10:17' '12/1/2010 10:18' '12/1/2010 10:20'
 '12/1/2010 10:21' '12/1/2010 10:22' '12/1/2010 10:23' '12/1/2010 10:24'
 '12/1/2010 10:26' '12/1/2010 10:27' '12/1/2010 10:28' '12/1/2010 10:29'
 '12/1/2010 10:30' '12/1/2010 10:31' '12/1/2010 10:32' '12/1/2010 10:33'
 '12/1/2010 10:35' '12/1/2010 10:36' '12/1/2010 10:37' '12/1/2010 10:38'
 '12/1/2010 10:39' '12/1/2010 10:40' '12/1/2010 10:42' '12/1/2010 10:44'
 '12/1/2010 10:46' '12/1/2010 10:47' '12/1/2010 10:48' '12/1/2010 10:49'
 '12/1/2010 10:50' '12/1/2010 10:51' '

- Eliminación de texto no correspondientes al formato de fechas : palabra 'Date'.

In [3813]:
# Eliminar las palabras 'Date:'
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].str.replace(r'Date:', '', regex=True).str.strip()
retail2_df = retail2_df.sort_values(by='InvoiceDate')
print(retail2_df['InvoiceDate'].unique())

['01-12-2010 09:47' '01-12-2010 10:09' '01-12-2010 11:27'
 '01-12-2010 13:42' '01/01/1900 00:00' '01/01/2050 00:00'
 '01/12/2010 25:61' '12/01/2010 09:08' '12/01/2010 10:41'
 '12/01/2010 10:52' '12/01/2010 12:22' '12/01/2010 14:55'
 '12/1/2010 10:00' '12/1/2010 10:01' '12/1/2010 10:03' '12/1/2010 10:04'
 '12/1/2010 10:05' '12/1/2010 10:06' '12/1/2010 10:07' '12/1/2010 10:08'
 '12/1/2010 10:10' '12/1/2010 10:12' '12/1/2010 10:13' '12/1/2010 10:15'
 '12/1/2010 10:16' '12/1/2010 10:17' '12/1/2010 10:18' '12/1/2010 10:20'
 '12/1/2010 10:21' '12/1/2010 10:22' '12/1/2010 10:23' '12/1/2010 10:24'
 '12/1/2010 10:26' '12/1/2010 10:27' '12/1/2010 10:28' '12/1/2010 10:29'
 '12/1/2010 10:30' '12/1/2010 10:31' '12/1/2010 10:32' '12/1/2010 10:33'
 '12/1/2010 10:35' '12/1/2010 10:36' '12/1/2010 10:37' '12/1/2010 10:38'
 '12/1/2010 10:39' '12/1/2010 10:40' '12/1/2010 10:42' '12/1/2010 10:44'
 '12/1/2010 10:46' '12/1/2010 10:47' '12/1/2010 10:48' '12/1/2010 10:49'
 '12/1/2010 10:50' '12/1/2010 10:51' '

- Convertir la columna InvoiceDate a datetime con el formato correcto que es dia, mes, año y hora. En este caso fueron las ventas correspondiente a un solo dia  12/01/2010 y con su respectiva hora

In [3814]:
# Convertir la columna InvoiceDate a datetime con el formato correcto
retail2_df['InvoiceDate'] = pd.to_datetime(retail2_df['InvoiceDate'], format='%d/%m/%Y %H:%M', errors='coerce')

# Ordenar el DataFrame por la columna InvoiceDate
retail2_df = retail2_df.sort_values(by='InvoiceDate')

# Convertir la columna InvoiceDate de nuevo a string en el formato deseado
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].dt.strftime('%d/%m/%Y %H:%M')

# Mostrar los valores únicos de la columna InvoiceDate
print(retail2_df['InvoiceDate'].unique())

['01/01/1900 00:00' '12/01/2010 08:26' '12/01/2010 08:28'
 '12/01/2010 08:34' '12/01/2010 08:35' '12/01/2010 09:00'
 '12/01/2010 09:01' '12/01/2010 09:02' '12/01/2010 09:03'
 '12/01/2010 09:06' '12/01/2010 09:07' '12/01/2010 09:08'
 '12/01/2010 09:09' '12/01/2010 09:11' '12/01/2010 09:12'
 '12/01/2010 09:13' '12/01/2010 09:14' '12/01/2010 09:15'
 '12/01/2010 09:16' '12/01/2010 09:17' '12/01/2010 09:19'
 '12/01/2010 09:21' '12/01/2010 09:22' '12/01/2010 09:23'
 '12/01/2010 09:24' '12/01/2010 09:25' '12/01/2010 09:26'
 '12/01/2010 09:27' '12/01/2010 09:28' '12/01/2010 09:29'
 '12/01/2010 09:30' '12/01/2010 09:31' '12/01/2010 09:32'
 '12/01/2010 09:33' '12/01/2010 09:34' '12/01/2010 09:35'
 '12/01/2010 09:36' '12/01/2010 09:38' '12/01/2010 09:39'
 '12/01/2010 09:40' '12/01/2010 09:41' '12/01/2010 09:42'
 '12/01/2010 09:43' '12/01/2010 09:44' '12/01/2010 09:45'
 '12/01/2010 09:46' '12/01/2010 09:48' '12/01/2010 09:49'
 '12/01/2010 09:50' '12/01/2010 09:51' '12/01/2010 09:52'
 '12/01/2010 0

- Defenición de fechas: En este caso fueron las ventas correspondiente a un solo dia  12/01/2010 y con su respectiva hora
- Verificación de datos inválidos

In [3815]:
# Intentar convertir la columna InvoiceDate a tipo datetime con el formato correcto
retail2_df['InvoiceDate'] = pd.to_datetime(retail2_df['InvoiceDate'], format='%d/%m/%Y %H:%M', errors='coerce')

# Definir la fecha válida
valid_date = pd.to_datetime('12/01/2010', format='%d/%m/%Y')

# Filtrar las fechas que no coinciden con la fecha válida (manteniendo la hora)
invalid_dates = retail2_df[retail2_df['InvoiceDate'].dt.date != valid_date.date()]

# Mostrar únicamente las columnas InvoiceNo y InvoiceDate de las filas con fechas inválidas
print(invalid_dates[['InvoiceNo', 'InvoiceDate']])

# Convertir la columna InvoiceDate de nuevo a string en el formato deseado
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].dt.strftime('%d/%m/%Y %H:%M')

# Contar las filas con fechas inválidas
invalid_count = invalid_dates.shape[0]
print(f"Cantidad de filas con fechas inválidas: {invalid_count}")

     InvoiceNo InvoiceDate
87      536677  1900-01-01
338     536380  1900-01-01
352     536473  1900-01-01
83      536598  1900-01-01
268     536636  1900-01-01
139     536687  1900-01-01
100     536588  1900-01-01
358     536425  1900-01-01
186     536407  1900-01-01
160     536544  1900-01-01
6       536707  2050-01-01
142     536374  2050-01-01
78           0  2050-01-01
308     536449  2050-01-01
104     536564  2050-01-01
173     536519  2050-01-01
241     536530  2050-01-01
380     536444  2050-01-01
279     536709  2050-01-01
34      536651  2050-01-01
69      536564  2050-01-01
223     536417         NaT
32      536439         NaT
114     536517         NaT
396     536652         NaT
412          0         NaT
52           0         NaT
174     536390         NaT
285     536669         NaT
318     536673         NaT
281     536630         NaT
90      536605         NaT
430     536370         NaT
58      536455         NaT
82      536432         NaT
105     536630         NaT
1

En el los datos inválidos encontramos fechas correspondiente a años como 2050, 1900, dias no correspondientes a la fecha a trabajar 30/02/2010, formatos de fechas no válidas como mes, dia, año y caracteres no correspondientes como -

## Correción de los datos invalidos mencionados anteriormente

- Cambio el formato de las fechas inválidas como caracteres

In [3816]:
# Asegurarse de que la columna InvoiceDate esté en formato string
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].astype(str)

# Cambiar el formato de las fechas en la columna InvoiceDate
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].str.replace('-', '/')
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].str.replace('01/12', '12/01')



# Ordenar el DataFrame por la columna InvoiceDate
retail2_df = retail2_df.sort_values(by='InvoiceDate')

# Mostrar los valores únicos de la columna InvoiceDate
print(retail2_df['InvoiceDate'].unique())

['01/01/1900 00:00' '01/01/2050 00:00' '12/01/2010 08:26'
 '12/01/2010 08:28' '12/01/2010 08:34' '12/01/2010 08:35'
 '12/01/2010 09:00' '12/01/2010 09:01' '12/01/2010 09:02'
 '12/01/2010 09:03' '12/01/2010 09:06' '12/01/2010 09:07'
 '12/01/2010 09:08' '12/01/2010 09:09' '12/01/2010 09:11'
 '12/01/2010 09:12' '12/01/2010 09:13' '12/01/2010 09:14'
 '12/01/2010 09:15' '12/01/2010 09:16' '12/01/2010 09:17'
 '12/01/2010 09:19' '12/01/2010 09:21' '12/01/2010 09:22'
 '12/01/2010 09:23' '12/01/2010 09:24' '12/01/2010 09:25'
 '12/01/2010 09:26' '12/01/2010 09:27' '12/01/2010 09:28'
 '12/01/2010 09:29' '12/01/2010 09:30' '12/01/2010 09:31'
 '12/01/2010 09:32' '12/01/2010 09:33' '12/01/2010 09:34'
 '12/01/2010 09:35' '12/01/2010 09:36' '12/01/2010 09:38'
 '12/01/2010 09:39' '12/01/2010 09:40' '12/01/2010 09:41'
 '12/01/2010 09:42' '12/01/2010 09:43' '12/01/2010 09:44'
 '12/01/2010 09:45' '12/01/2010 09:46' '12/01/2010 09:48'
 '12/01/2010 09:49' '12/01/2010 09:50' '12/01/2010 09:51'
 '12/01/2010 0

In [3817]:
print(retail2_df[['InvoiceNo','InvoiceDate']])

     InvoiceNo       InvoiceDate
87      536677  01/01/1900 00:00
186     536407  01/01/1900 00:00
358     536425  01/01/1900 00:00
100     536588  01/01/1900 00:00
139     536687  01/01/1900 00:00
160     536544  01/01/1900 00:00
83      536598  01/01/1900 00:00
352     536473  01/01/1900 00:00
338     536380  01/01/1900 00:00
268     536636  01/01/1900 00:00
69      536564  01/01/2050 00:00
78           0  01/01/2050 00:00
308     536449  01/01/2050 00:00
104     536564  01/01/2050 00:00
173     536519  01/01/2050 00:00
241     536530  01/01/2050 00:00
380     536444  01/01/2050 00:00
279     536709  01/01/2050 00:00
34      536651  01/01/2050 00:00
142     536374  01/01/2050 00:00
6       536707  01/01/2050 00:00
307     536365  12/01/2010 08:26
313     536365  12/01/2010 08:26
175     536365  12/01/2010 08:26
38      536365  12/01/2010 08:26
47      536365  12/01/2010 08:26
361     536365  12/01/2010 08:26
302     536365  12/01/2010 08:26
31      536366  12/01/2010 08:28
211     53

In [3818]:
# Convertimos la columna InvoiceDate a datetime temporalmente para manejar los valores válidos
retail2_df['InvoiceDate'] = pd.to_datetime(retail2_df['InvoiceDate'], errors='coerce', dayfirst=True)

# Guardamos las fechas originales para las facturas sin número de factura
original_dates = retail2_df[retail2_df['InvoiceNo'] == 0]['InvoiceDate']

# Ordenamos el DataFrame por InvoiceNo para asegurar que las facturas estén en orden
retail2_df = retail2_df.sort_values(by='InvoiceNo')

# Iteramos sobre las filas para corregir los valores inválidos
for i in range(1, len(retail2_df)):
    if (pd.isnull(retail2_df.loc[retail2_df.index[i], 'InvoiceDate']) or 
        retail2_df.loc[retail2_df.index[i], 'InvoiceDate'].year in [1900, 2050]) and \
        retail2_df.loc[retail2_df.index[i], 'InvoiceNo'] != 0:
        
        previous_date = retail2_df.loc[retail2_df.index[i-1], 'InvoiceDate']
        new_date = previous_date + timedelta(minutes=1)
        retail2_df.loc[retail2_df.index[i], 'InvoiceDate'] = new_date

# Aseguramos que las facturas duplicadas tengan la misma hora
retail2_df['InvoiceDate'] = retail2_df.groupby('InvoiceNo')['InvoiceDate'].transform('first')

# Restauramos las fechas originales para las facturas sin número de factura
retail2_df.loc[retail2_df['InvoiceNo'] == 0, 'InvoiceDate'] = original_dates

# Convertimos de nuevo a string para mantener el formato original sin segundos
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].dt.strftime('%d/%m/%Y %H:%M')

# Ordenamos el DataFrame por fecha y hora
retail2_df = retail2_df.sort_values(by='InvoiceDate')

# Imprime el resultado de las dos columnas para verificar
print(retail2_df[['InvoiceNo', 'InvoiceDate']])

     InvoiceNo       InvoiceDate
78           0  01/01/2050 00:00
307     536365  12/01/2010 08:26
302     536365  12/01/2010 08:26
47      536365  12/01/2010 08:26
175     536365  12/01/2010 08:26
313     536365  12/01/2010 08:26
38      536365  12/01/2010 08:26
361     536365  12/01/2010 08:26
211     536366  12/01/2010 08:28
31      536366  12/01/2010 08:28
273     536367  12/01/2010 08:34
222     536367  12/01/2010 08:34
282     536367  12/01/2010 08:34
86      536367  12/01/2010 08:34
247     536367  12/01/2010 08:34
388     536367  12/01/2010 08:34
230     536367  12/01/2010 08:34
206     536367  12/01/2010 08:34
360     536367  12/01/2010 08:34
117     536367  12/01/2010 08:34
438     536368  12/01/2010 08:35
72      536368  12/01/2010 08:35
421     536369  12/01/2010 08:36
374     536370  12/01/2010 09:00
267     536370  12/01/2010 09:00
430     536370  12/01/2010 09:00
341     536371  12/01/2010 09:01
383     536371  12/01/2010 09:01
10      536372  12/01/2010 09:02
30      53

In [3819]:
# Imprimir los datos de las columnas especificadas en el índice 312
print(retail2_df.loc[251, ['InvoiceNo', 'Description', 'InvoiceDate']])

InvoiceNo                              0
Description    HAND WARMER RED POLKA DOT
InvoiceDate             12/01/2010 12:53
Name: 251, dtype: object


In [3820]:
# Imprimir los datos de las columnas especificadas en el índice 312
print(retail2_df.loc[208, ['InvoiceNo', 'Description', 'InvoiceDate']])

InvoiceNo                                       0
Description    WHITE HANGING HEART T-LIGHT HOLDER
InvoiceDate                      12/01/2010 13:15
Name: 208, dtype: object


In [3821]:
# Convertimos la columna InvoiceDate a datetime
retail2_df['InvoiceDate'] = pd.to_datetime(retail2_df['InvoiceDate'], format='%d/%m/%Y %H:%M')

# Creamos un diccionario para asignar los valores correctos basados en el índice
index_mapping = {
    
    251: {'InvoiceNo1': 536603, 'InvoiceDate': '12/01/2010 12:53'},
    104: {'InvoiceNo1': 536602, 'InvoiceDate': '12/01/2010 12:52'}
    
}

# Asignamos los valores correctos basados en el índice
for idx, values in index_mapping.items():
    retail2_df.loc[retail2_df.index == idx, 'InvoiceNo'] = values['InvoiceNo1']
    retail2_df.loc[retail2_df.index == idx, 'InvoiceDate'] = pd.to_datetime(values['InvoiceDate'], format='%d/%m/%Y %H:%M')

# Convertimos de nuevo a string para mantener el formato original sin segundos
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].dt.strftime('%d/%m/%Y %H:%M')

# Ordenamos el DataFrame por número de factura
retail2_df = retail2_df.sort_values(by='InvoiceNo')

In [3822]:
# Convertimos la columna InvoiceDate a datetime temporalmente para manejar los valores válidos
retail2_df['InvoiceDate'] = pd.to_datetime(retail2_df['InvoiceDate'], errors='coerce', dayfirst=True)

# Ordenamos el DataFrame por fecha y hora para asegurar que las fechas estén en orden
retail2_df = retail2_df.sort_values(by='InvoiceDate')

# Inicializamos el número de factura
current_invoice_no = int(retail2_df['InvoiceNo'].max()) + 1

# Iteramos sobre las filas para asignar números de factura a las filas sin número de factura
for i in range(len(retail2_df)):
    if retail2_df.loc[retail2_df.index[i], 'InvoiceNo'] == 0:
        previous_invoice_no = retail2_df.loc[retail2_df.index[i-1], 'InvoiceNo']
        retail2_df.loc[retail2_df.index[i], 'InvoiceNo'] = previous_invoice_no + 1

# Convertimos de nuevo a string para mantener el formato original sin segundos
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].dt.strftime('%d/%m/%Y %H:%M')

# Ordenamos el DataFrame por fecha y hora
retail2_df = retail2_df.sort_values(by='InvoiceDate')

# Imprime el resultado de las dos columnas para verificar
print(retail2_df[['InvoiceNo', 'InvoiceDate']])

     InvoiceNo       InvoiceDate
78      536745  01/01/2050 00:00
302     536365  12/01/2010 08:26
175     536365  12/01/2010 08:26
313     536365  12/01/2010 08:26
47      536365  12/01/2010 08:26
307     536365  12/01/2010 08:26
38      536365  12/01/2010 08:26
361     536365  12/01/2010 08:26
211     536366  12/01/2010 08:28
31      536366  12/01/2010 08:28
117     536367  12/01/2010 08:34
230     536367  12/01/2010 08:34
388     536367  12/01/2010 08:34
222     536367  12/01/2010 08:34
206     536367  12/01/2010 08:34
360     536367  12/01/2010 08:34
273     536367  12/01/2010 08:34
282     536367  12/01/2010 08:34
247     536367  12/01/2010 08:34
86      536367  12/01/2010 08:34
72      536368  12/01/2010 08:35
438     536368  12/01/2010 08:35
421     536369  12/01/2010 08:36
267     536370  12/01/2010 09:00
374     536370  12/01/2010 09:00
430     536370  12/01/2010 09:00
383     536371  12/01/2010 09:01
341     536371  12/01/2010 09:01
10      536372  12/01/2010 09:02
30      53

In [3823]:
# Imprimir los datos de las columnas especificadas en el índice 312
print(retail2_df.loc[104, ['InvoiceNo', 'Description', 'InvoiceDate']])

InvoiceNo                                  536602
Description    WHITE HANGING HEART T-LIGHT HOLDER
InvoiceDate                      12/01/2010 12:52
Name: 104, dtype: object


- Cambio de formatos de fechas

In [3824]:
# Convertimos la columna InvoiceDate a datetime
retail2_df['InvoiceDate'] = pd.to_datetime(retail2_df['InvoiceDate'], format='%d/%m/%Y %H:%M')

# Creamos un diccionario para asignar los valores correctos basados en el índice
index_mapping = {
    78: {'InvoiceNo1': 536653, 'InvoiceDate': '12/01/2010 13:43'},
    52: {'InvoiceNo1': 536730, 'InvoiceDate': '12/01/2010 15:00'},
    412: {'InvoiceNo1': 536384, 'InvoiceDate': '12/01/2010 9:14'}
}

# Asignamos los valores correctos basados en el índice
for idx, values in index_mapping.items():
    retail2_df.loc[retail2_df.index == idx, 'InvoiceNo'] = values['InvoiceNo1']
    retail2_df.loc[retail2_df.index == idx, 'InvoiceDate'] = pd.to_datetime(values['InvoiceDate'], format='%d/%m/%Y %H:%M')

# Convertimos de nuevo a string para mantener el formato original sin segundos
retail2_df['InvoiceDate'] = retail2_df['InvoiceDate'].dt.strftime('%d/%m/%Y %H:%M')

# Ordenamos el DataFrame por número de factura
retail2_df = retail2_df.sort_values(by='InvoiceNo')


In [3825]:
# Identificar los datos inválidos en la columna Description, incluyendo NaN y vacíos
invalid_data = retail2_df[retail2_df['InvoiceDate'].isnull() | (retail2_df['InvoiceDate'] == '') | (retail2_df['InvoiceDate'].str.lower() == 'nan')]

# Mostrar los datos inválidos y la cantidad
print(invalid_data[['InvoiceNo','InvoiceDate']])
print(f"Cantidad de datos inválidos: {invalid_data.shape[0]}")

Empty DataFrame
Columns: [InvoiceNo, InvoiceDate]
Index: []
Cantidad de datos inválidos: 0


In [3826]:
# Imprimir los datos de las columnas especificadas en el índice 312
print(retail2_df.loc[251, ['InvoiceNo', 'Description', 'InvoiceDate']])

InvoiceNo                         536603
Description    HAND WARMER RED POLKA DOT
InvoiceDate             12/01/2010 12:53
Name: 251, dtype: object


## Continuacion de la columna InvoiceCode
- Verificación del rango del número de los datos de la columna : Número de factura que empieza hasta que se termina.


- verificacion del rango del número de los datos de la columna 

In [3827]:
# Suponiendo que retail2_df ya está definido
min_invoice = retail2_df['InvoiceNo'].min()
max_invoice = retail2_df['InvoiceNo'].max()

print(f"El número de factura empieza en: {min_invoice}")
print(f"El número de factura finaliza en: {max_invoice}")

El número de factura empieza en: 536365
El número de factura finaliza en: 536744


- Verificación de facturas faltantes entre los rangos  

In [3828]:
expected_range = set(range(536365, 536744))
actual_values = set(retail2_df['InvoiceNo'].dropna().astype(int))
missing_values = expected_range - actual_values

print(f"Valores faltantes en la columna InvoiceNo: {sorted(missing_values)}")

Valores faltantes en la columna InvoiceNo: []


In [3829]:
# Ordenamos el DataFrame por fecha y hora
retail2_df = retail2_df.sort_values(by='InvoiceDate')

# Imprime el resultado de las dos columnas para verificar
print(retail2_df[['InvoiceNo', 'InvoiceDate']])

     InvoiceNo       InvoiceDate
302     536365  12/01/2010 08:26
175     536365  12/01/2010 08:26
313     536365  12/01/2010 08:26
47      536365  12/01/2010 08:26
307     536365  12/01/2010 08:26
38      536365  12/01/2010 08:26
361     536365  12/01/2010 08:26
211     536366  12/01/2010 08:28
31      536366  12/01/2010 08:28
273     536367  12/01/2010 08:34
282     536367  12/01/2010 08:34
117     536367  12/01/2010 08:34
360     536367  12/01/2010 08:34
206     536367  12/01/2010 08:34
247     536367  12/01/2010 08:34
388     536367  12/01/2010 08:34
222     536367  12/01/2010 08:34
86      536367  12/01/2010 08:34
230     536367  12/01/2010 08:34
72      536368  12/01/2010 08:35
438     536368  12/01/2010 08:35
421     536369  12/01/2010 08:36
267     536370  12/01/2010 09:00
430     536370  12/01/2010 09:00
374     536370  12/01/2010 09:00
341     536371  12/01/2010 09:01
383     536371  12/01/2010 09:01
10      536372  12/01/2010 09:02
30      536373  12/01/2010 09:03
142     53